In [54]:
# Magic to automatically update imports if functions in utils are changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
from sklearn.model_selection import train_test_split

stores_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
bus_routes = pd.read_csv("data/busstops_norway.csv")
#stores_train, stores_val = train_test_split(stores_train, test_size=0.2, random_state=0)

In [77]:
bus_routes.head()

,busstop_id,stopplace_type,importance_level,side_placement,geometry
0,853cb081-cc32-4880-aa3e-26e96870d874,Plattform og lomme,Mangler viktighetsnivå,LEFT_AND_RIGHT,POINT(6.54064723379766 61.0618887850553)
1,156b052b-2771-497a-b4f4-97fed59e1aca,"Lomme og skilt, ikke plattform",Mangler viktighetsnivå,LEFT_AND_RIGHT,POINT(5.89980086113255 60.1421872817075)
2,7312a280-e14f-4b09-a421-02e8fe1bc63e,NaN,Mangler viktighetsnivå,MIDDLE_LEFT,POINT(10.7781327278563 59.9299988828761)
3,d9cda2c7-355a-49c1-b56c-a33180d2a82e,NaN,Mangler viktighetsnivå,MIDDLE_LEFT,POINT(10.7781496457324 59.9301044643692)
4,f803bcd3-182d-450b-bbb4-113c6ca885c2,NaN,Mangler viktighetsnivå,MIDDLE_RIGHT,POINT(10.7781676444854 59.9302099995898)


In [78]:
stores_train.head()

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528


In [79]:
bus_routes["point"] = bus_routes.geometry.apply(lambda x: np.array(x[6:-1].split(" ")).astype(float)[::-1])

In [80]:
bus_routes["point"].sample(5)

23146     [59.4624847948068, 5.2753746482926]
31885    [59.7753207451503, 9.88703888682864]
65285    [58.5606589255191, 8.90257899104042]
3452     [59.4383887781571, 10.6297822994633]
9679     [66.9223509333203, 14.7726150906538]
Name: point, dtype: object

In [81]:
from scipy.spatial.distance import cdist


def closest_point(point, points, n_closest = 3, _sum = False):
    """ Find closest point from a list of points. """
    if(len(points) == 0):
        return None
    dist_points = cdist([point], points)
    dist_points = dist_points.flatten()
    dist_points.sort()
    n_closest_dist = dist_points[:n_closest]
    if(_sum):
        return np.sum(n_closest_dist)
    return np.mean(n_closest_dist)

In [82]:
stores_train["point"] = [(x, y) for x,y in zip(stores_train['lat'], stores_train['lon'])]

In [83]:
stores_train['mean_3_closest_bus_stop'] = [closest_point(x["point"], list(bus_routes["point"])) for _, x in stores_train.iterrows()]

In [84]:
stores_train["mean_3_closest_bus_stop"].describe()

count    12859.000000
mean         0.004861
std          0.015264
min          0.000231
25%          0.001614
50%          0.002499
75%          0.004204
max          0.499380
Name: mean_3_closest_bus_stop, dtype: float64

In [85]:
stores_train['mean_1_closest_bus_stop'] = [closest_point(x["point"], list(bus_routes["point"]), 1) for _, x in stores_train.iterrows()]
stores_train['mean_5_closest_bus_stop'] = [closest_point(x["point"], list(bus_routes["point"]), 5) for _, x in stores_train.iterrows()]
stores_train['mean_7_closest_bus_stop'] = [closest_point(x["point"], list(bus_routes["point"]), 7) for _, x in stores_train.iterrows()]

In [86]:
stores_train['sum_3_closest_bus_stop'] = [closest_point(x["point"], list(bus_routes["point"]), 3, _sum=True) for _, x in stores_train.iterrows()]
stores_train['sum_5_closest_bus_stop'] = [closest_point(x["point"], list(bus_routes["point"]), 5, _sum=True) for _, x in stores_train.iterrows()]
stores_train['sum_7_closest_bus_stop'] = [closest_point(x["point"], list(bus_routes["point"]), 7, _sum=True) for _, x in stores_train.iterrows()]

In [ ]:
stores_train.head()

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,point,mean_3_closest_bus_stop,mean_1_closest_bus_stop,mean_5_closest_bus_stop,mean_7_closest_bus_stop,sum_3_closest_bus_stop,sum_5_closest_bus_stop,sum_7_closest_bus_stop
2506,990857423-990974489-88185,2016,BAKER KLAUSEN UNION,1.2.2.0,Bakery shop,6020601,GRØNLAND 54,59.743135,10.194627,BAKER KLAUSEN DRAMMEN,NaN,4.447,"(59.74313530386021, 10.1946270640764)",0.002881,0.002032,0.003508,0.004101,0.008643,0.017540,0.028706
12305,948879220-972145203-18800,2016,PRIKKEN RENS AS,3.4.2.0,Laundromats and dry cleaners,2350203,ENERGIVEGEN 5,60.149288,11.187290,NaN,NaN,2.040,"(60.14928769290111, 11.1872899024035)",0.007244,0.002121,0.008454,0.009322,0.021733,0.042271,0.065255
4142,951136751-971668970-6795,2016,EXPERT HOKKSUND,2.3.1.0,Electronics stores,6240307,SUNDMOEN NÆRINGSOMRÅDE,59.768986,9.899066,EXPERT,NaN,36.489,"(59.7689864010885, 9.89906598218001)",0.001622,0.001458,0.001887,0.002409,0.004865,0.009434,0.016863
8561,980237508-980246167-71346,2016,INVIT AS,2.9.1.0,Gifts and interior design shops,15040206,APOTEKERGATA 9,62.471607,6.151301,NaN,NaN,5.739,"(62.4716074153893, 6.15130082978494)",0.005224,0.002676,0.008216,0.011220,0.015671,0.041079,0.078540
818,974421305-974489902-136560,2016,MUCHO MAS,1.1.6.2,Restaurants,3012312,THORVALD MEYERS GATE 36,59.924205,10.759401,NaN,NaN,15.573,"(59.9242046792076, 10.759400799707802)",0.001884,0.001102,0.002545,0.003008,0.005653,0.012724,0.021053


In [ ]:
stores_train = stores_train[[
    'store_id', 
    'mean_1_closest_bus_stop', 
    'mean_3_closest_bus_stop', 
    'mean_5_closest_bus_stop', 
    'mean_7_closest_bus_stop', 
    'sum_3_closest_bus_stop',
    'sum_5_closest_bus_stop',
    'sum_7_closest_bus_stop',
    ]]

In [ ]:
stores_train.head()

,store_id,mean_1_closest_bus_stop,mean_3_closest_bus_stop,mean_5_closest_bus_stop,mean_7_closest_bus_stop,sum_3_closest_bus_stop,sum_5_closest_bus_stop,sum_7_closest_bus_stop
2506,990857423-990974489-88185,0.002032,0.002881,0.003508,0.004101,0.008643,0.017540,0.028706
12305,948879220-972145203-18800,0.002121,0.007244,0.008454,0.009322,0.021733,0.042271,0.065255
4142,951136751-971668970-6795,0.001458,0.001622,0.001887,0.002409,0.004865,0.009434,0.016863
8561,980237508-980246167-71346,0.002676,0.005224,0.008216,0.011220,0.015671,0.041079,0.078540
818,974421305-974489902-136560,0.001102,0.001884,0.002545,0.003008,0.005653,0.012724,0.021053


In [ ]:
stores_train.to_csv(path_or_buf="temp_data/closest_bus_stops.csv", index=False)